# Web Scraping with Python

#### Challenge: 

We work in an importer and the price of our products is linked to a quotation from:
- Dollar
- Euro
- Gold

We need to find this information from the internet, automatically, the quotation from this 3 itens and based on that how much we should charge for our products, considering a contribution margin that we have in our database.

DataBase: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

To solve this problem, we're going to create a Web Scraping program

- Using selenium

In [2]:
!pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

# Step 1: Dollar quotation
navegador.get('https://www.google.com.br/')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# Step 2: Euro quotation
navegador.get('https://www.google.com.br/')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# Step 3: Gold quotation
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')

print(cotacao_ouro)

navegador.quit()



5.1133
5.2124999999999995
296.61


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [4]:
# Step 4: Update the quotations in our databse
import pandas as pd

tabela_produtos = pd.read_excel('Produtos.xlsx')
tabela_produtos

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Updating prices and Preço Final calculation

In [5]:
# Step 5: Recalculate item prices
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)
                    
# Updating actual base prices (original base price * quote)
tabela_produtos['Preço de Compra'] = tabela_produtos['Preço Original'] * tabela_produtos['Cotação']

# Updating final purchase prices (purchase price * margin)
tabela_produtos['Preço de Venda'] = tabela_produtos['Preço de Compra'] * tabela_produtos['Margem']                   
                    
tabela_produtos

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.1133,5113.248867,1.40,7158.548414
1,Carro Renault,4500.00,Euro,5.2125,23456.250000,2.00,46912.500000
2,Notebook Dell,899.99,Dólar,5.1133,4601.918867,1.70,7823.262074
3,IPhone,799.00,Dólar,5.1133,4085.526700,1.70,6945.395390
4,Carro Fiat,3000.00,Euro,5.2125,15637.500000,1.90,29711.250000
5,Celular Xiaomi,480.48,Dólar,5.1133,2456.838384,2.00,4913.676768
6,Joia 20g,20.00,Ouro,296.6100,5932.200000,1.15,6822.030000


### Now let's export the new updated price base

In [6]:
# Step 6: Exporting to the databse

tabela_produtos.to_excel('Produtos_Atualizado.xlsx', index= False)